# Feedback Alignment

##### Imports and helper functions

In [10]:
import numpy as np
import torch as th
import matplotlib.pyplot as plt
import torchvision
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [11]:
def get_loaders(batch_size, fashion=False):
    mnist = torchvision.datasets.MNIST
    if fashion:
        mnist = torchvision.datasets.FashionMNIST

    transform = torchvision.transforms.Compose(
        [torchvision.transforms.ToTensor(),])

    trainloader = th.utils.data.DataLoader(
        mnist(root="./data", train=True, download=True, transform=transform),
        batch_size=batch_size,
        shuffle=True,
        num_workers=2)
    testloader = th.utils.data.DataLoader(
        mnist(root="./data", train=False, download=True, transform=transform),
        batch_size=batch_size,
        shuffle=False,
        num_workers=2)
    return trainloader, testloader

## Linear function approximation

<img href="images/linear.png">
We are considering a three-layer network of linear neurons, shown above. The network's output is $\boldsymbol{y}=W\boldsymbol{h}$, where $\boldsymbol{h}$ is the hidden-unit activity vector, given by $\boldsymbol{h}=W_0\boldsymbol{x}$, where $\boldsymbol{x}$ is the input to the network. $W_0$ is the matrix of synaptic weights from $\boldsymbol{x}$ to $\boldsymbol{h}$, and $W$ is the weights from $\boldsymbol{h}$ to $\boldsymbol{y}$. The network learns to approximate a linear function, $T$ (for 'target'). It's goal is to reduce the squared error, or loss, $\mathcal{L}=\frac{1}{2}\boldsymbol{e}^{T}\boldsymbol{e}$, where the error $\boldsymbol{e}=\boldsymbol{y^*}-\boldsymbol{y}=T\boldsymbol{x}-\boldsymbol{y}$. To train this network, the feedback alignment algorithm adjusts $W$ in the same way as backprop, i.e. $\Delta W\propto\frac{\partial\mathcal{L}}{\partial W}=-\boldsymbol{e}\boldsymbol{h}^T$, but for $W_0$, it uses a simpler formula, which needs no information about $W$ or any other synapses, but instead, sends $\boldsymbol{e}$ through a fixed random matrix $B$:
$$\Delta W_0\propto B\boldsymbol{e}\boldsymbol{x}^T$$

### Data Generation

The target linear function $T$ maps vectors from a $30$- to a $10$-dimensional space, i.e. $T$ has a shape of $10\times 30$. The elements of $T$ are drawn at random, that is, uniformly, from the range $[-1,1]$. Once chosen, the target matrix is fixed, so that each algorithm (i.e. feedback alignment and backpropagation) tried to learn the same function. Moreover, all algorithms are trained on the same sequence of input/output pairs, with $x\sim{}\mathcal{N}(\mu=0,\Sigma=I)$, $y^*=Tx$. We have chosen the **number of inputs** to be **100**.

##### linear function, inputs, and output generation

In [12]:
num_inputs = 100

T = np.random.uniform(low=-1.0, high=1.0, size=(10, 30))
x_data = np.random.randn(30, num_inputs)
y_data = T @ x_data

`x_data`: `30 x num_inputs`

`y_data`: `10 x num_inputs`

##### Weights and biases random initalization

The elements of $B$ are drawn from the uniform distribution over $[-0.5, 0.5)$, while the elements of the network weight matrix, $W_0$ and $W$, are drawn unifromly from the range $[-0.01,0.01)$.

In [13]:
a = 0.01
W0 = np.random.uniform(-a, a, (20, 30))
b0 = np.random.uniform(-a, a, 20)

W = np.random.uniform(-a, a, (10, 20))
b = np.random.uniform(-a, a, 10)

a = 0.5
B = np.random.uniform(-a, a, (20, 10))

##### Training and Test dataset preparation

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x_data.T, y_data.T, test_size=0.25, shuffle=True)
x_train, x_test, y_train, y_test = x_train.T, x_test.T, y_train.T, y_test.T

In [15]:
x_test.shape

(30, 25)

##### Hyperparameters

In [16]:
eta = 0.1
num_epochs = 1000
batch_size = 20
num_batches = x_train.shape[1] / batch_size

##### Creating batches

In [17]:
x_train_batches, y_train_batches = np.split(x_train, num_batches, axis=1), np.split(y_train, num_batches, axis=1)

In [18]:
for epoch in range(num_epochs):
    
    delta_W0, delta_W, loss = 0, 0, 0
    for i in range(len(x_train_batches)):
        training_data, training_labels = x_train_batches[i], y_train_batches[i]
        
        h = W0 @ training_data
        y = W @ h
        
        e = training_labels - y
        loss += 0.5 * np.square(np.linalg.norm(e))
        
        delta_BP = W.T @ e
        delta_FA = B @ e
        
        delta_W = delta_W + e @ h.T
        delta_W0 = delta_W0 + B @ e @ training_data.T
    
    W = W + (eta / (x_train.shape[1])) * delta_W
    W0 = W0 + (eta / (x_train.shape[1])) * delta_W0
    if epoch % 500 == 0:
        print(f"      Epoch {epoch}")
        print(f"      Current loss: {loss}")

      Epoch 0
      Current loss: 4063.639471419209
      Epoch 500
      Current loss: 4.54657687026908e-05


In [19]:
h = W0 @ x_test
y = W @ h
e = y_test - y
test_error = 0.5 * np.square(np.linalg.norm(e))
test_error

6.825886652188656e-09

## Non-Linear function approximation

##### Hyperparameters

In [38]:
eta = 0.001
num_epochs = 5
batch_size = 20
num_batches = (60000) / batch_size

##### Weights and biases random initalization

In [39]:
a = 0.01
W0 = np.random.uniform(-a, a, (1000, 784))
b0 = np.random.uniform(-a, a, 1000)

W = np.random.uniform(-a, a, (10, 1000))
b = np.random.uniform(-a, a, 10)

a = 0.5
B = np.random.uniform(-a, a, (1000, 10))

In [40]:
u, s, vh = np.linalg.svd(W0)

##### Training and Test dataset preparation

In [41]:
 trainloader, testloader = get_loaders(batch_size)

In [42]:
def sigma(x):
    return 1 / (1 + np.exp(-x))

In [43]:
for epoch in range(num_epochs):
    delta_W0, delta_W, loss, size = 0, 0, 0, 0
    for training_data, training_labels in tqdm(trainloader, desc=f"      Epoch {epoch}"):
        training_data, training_labels = training_data.view(-1, 784).numpy(), training_labels.numpy()
        training_labels = np.reshape(training_labels, newshape=-1)
        training_labels = np.eye(10)[training_labels].T
        
        h = sigma(W0 @ training_data.T)
        h = h / np.linalg.norm(h)
        y = sigma(W @ h)
        e = training_labels - y
        loss += 0.5 * np.square(np.linalg.norm(e))
        
        delta_BP = W.T @ e
        delta_FA = B @ e
        
#         one_y, one_h = np.ones_like(y), np.ones_like(h)
        # y' = y * (1 - y)
        # h' = h * (1 - h)
        
        delta_W = delta_W + (e * (y * (1 - y))) @ h.T
        delta_W0 = delta_W0 + (delta_FA * h * (1 - h)) @ training_data
        
        size += training_data.shape[0]
    
    W = W + eta * delta_W
    W0 = W0 + eta * delta_W0
    if epoch % 10 == 0:
        print(f"      Epoch {epoch}")
        print(f"      Current loss: {loss}")

      Epoch 1:   0%|          | 0/3000 [00:00<?, ?it/s]

      Epoch 0
      Current loss: 74973.11444391057


      Epoch 4: 100%|██████████| 3000/3000 [00:29<00:00, 101.28it/s]


## 4-Layer Non-Linear function approximation

In [44]:
loss

39282.43509643624